# TRIGGER Demo

[Referenz: Trigger](https:\learn.microsoft.com\en-us\sql\t-sql\statements\create-trigger-transact-sql?view=sql-server-ver16)  

**Idee**: Beim Eintragen eines Datensatzes in die **Arbeitet\_An Tabelle** soll geprüft werden, ob das **Projekt-Budget** noch ausreicht.

- Neue Tätigkeiten sollen nur dann eingefügt werden, wenn der Mitarbeiter nicht bereits für dieses Projekt eingetragen ist.
- Tätigkeiten müssen in Arbeitet\_An einzeln eingetragen werden. 
- Nur bei ausreichendem Budget erfolgt der Eintrag.

Dazu wird die Projekttabelle erweitert um die Spalten **Personalbudget** und **Personalkostenstand.**

**Challenge:** Die Tabelle Arbeitet\_An benötigt außerdem einen **Update-Trigger**, der beim Aufstocken der Stunden eines Mitarbeiters prüft, ob das Budget ausreicht.

In [ ]:
USE NormalisierungsDemo;

In [ ]:
-- In Tabelle Projekt Spalten Personalbudget und Personalkostenstand hinzufügen
ALTER TABLE Projekt
ADD Personalbudget DECIMAL(9,2) CONSTRAINT df_Personalbudget DEFAULT 10000,
    Personalkostenstand DECIMAL(9,2) CONSTRAINT df_Personalkostenstand DEFAULT 0;

In [ ]:
-- Die Standardwerte für Personalbudget und Personalkostenstand auf zuvor vorhandene Projekte anwenden
UPDATE Projekt
SET Personalbudget = 10000,
    Personalkostenstand = 0;

In [ ]:
-- Den Stand der Personalkosten aktualisieren per Funktionsaufruf
UPDATE Projekt
SET Personalkostenstand = dbo.fnc_Personalkosten(Beschreibung);

In [ ]:
-- Trigger erstellen
CREATE OR ALTER TRIGGER dbo.tr_Kostenbremse
ON dbo.Arbeitet_An
INSTEAD OF INSERT
AS
BEGIN
	SET NOCOUNT ON;
    BEGIN TRY
        BEGIN TRANSACTION;
        -- Verfügbares Büdget für Projekt ermitteln
        DECLARE @Verfügbar DECIMAL(9,2), @Kosten DECIMAL(9,2);
        DECLARE @Restbudget DECIMAL(9,2), @Projekt INT, @Test INT;
        DECLARE @Posten AS TABLE (
                PersonalNr INT NOT NULL,
                ProjNr INT NOT NULL,
                TätigkeitsNr INT NOT NULL,
                Stunden INT NOT NULL
            );
        INSERT INTO @Posten (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
        SELECT  PersonalNr, ProjNr, TätigkeitsNr, Stunden
        FROM inserted;
        -- Datensätze müssen einzeln in Arbeitet_An eingefügt werden
        SELECT @Test = COUNT(*) FROM @Posten;
        IF @Test > 1 THROW 50000, 'Fehler: Datensätze müssen einzeln eigefügt werden!', 1;
        -- PK Verletzung prüfen: Die Kombination PersonalNr+MitarbeiterNr muss eindeutig sein
        SELECT @Test = COUNT(*)
            FROM Arbeitet_An AS aa
            JOIN @Posten AS po ON aa.PersonalNr = po.PersonalNr
                AND aa.ProjNr = po.ProjNr;
        -- SELECT @Test AS Test;
        IF @Test > 0 THROW 50000, 'Der Mitarbeiter ist bereits für dieses Projekt registriert!', 1;
        -- Kosten für neue Projektstunden ermitteln
        SELECT @Kosten = (t.Stundenlohn * po.Stunden)
            FROM @Posten AS po
            JOIN Tätigkeit AS t ON po.TätigkeitsNr = t.TätigkeitsNr;
        -- SELECT @Kosten As Kosten;
        -- Budget ermitteln
        SELECT @Restbudget = (pr.Personalbudget - pr.Personalkostenstand)
            FROM Projekt AS pr
            JOIN @Posten AS po ON pr.ProjNr = po.ProjNr;
        -- SELECT @Restbudget AS Restbudget, @Kosten AS Kosten;
        IF @Kosten > @Restbudget THROW 50000, 'Kein ausreichendes Budget verfügbar!', 1;
        -- Bei verfügbarem Budget Arbeitet_An Datensatz schreiben und Projektkostenstand aktualisieren
        -- Projektkostenstand aktualisieren
        UPDATE Projekt 
            SET Personalkostenstand += @Kosten
            WHERE ProjNr = (SELECT ProjNr FROM @Posten);
         -- Arbeitet_An Datensatz einfügen   
        INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr, Stunden)
            SELECT PersonalNr, ProjNr, TätigkeitsNr, Stunden
            FROM @Posten;
        PRINT 'Yay! • Tätigkeit registriert. • Kostenstand aktualisiert.';
        COMMIT TRANSACTION;
    END TRY
    BEGIN CATCH
        ROLLBACK TRANSACTION;
        THROW;
    END CATCH;
    SET NOCOUNT OFF;
END;

In [ ]:
-- Testfall: Fehler beim einfügen mehrerer Datensätze
INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr,Stunden)
VALUES(5,1,2,100),
    (5,2,2,100);

In [ ]:
-- Testfall: Fehler, da Mitarbeiter schon im Projekt ist (PK Verletzung)
INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr,Stunden)
VALUES(2,1,1,100);

In [ ]:
-- Testfall: Sollte EINMAL funktionieren
INSERT INTO Arbeitet_An (PersonalNr, ProjNr, TätigkeitsNr,Stunden)
VALUES(5,1,2,100);

In [ ]:
-- Abfrage zur Kontrolle: Personalkostenstand mit Kosten aus Tätigkeiten vergleichen
SELECT p.Beschreibung AS Projekt, p.Personalbudget , p.Personalkostenstand, SUM(1.0 * aa.Stunden * t.Stundenlohn) AS [Kostensumme aus Details]
FROM Projekt AS P 
LEFT JOIN Arbeitet_An AS aa ON p.ProjNr = aa.ProjNr
LEFT JOIN Tätigkeit AS t ON aa.TätigkeitsNr = t.TätigkeitsNr
GROUP BY p.Beschreibung, p.Personalbudget, p.Personalkostenstand;

In [ ]:
-- Trigger löschen
DROP TRIGGER dbo.tr_Kostenbremse;